# Experiment 01-01 : Create a simple SHA256 Hash using OpenCL

In [409]:
import sys
sys.path.append('../../')

In [410]:
import numpy as np
import hashlib
from binascii import hexlify, unhexlify
import pyopencl as cl
from Library.opencl_information import opencl_information


## Show the available OpenCL Platforms

In [411]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Sep 30 2022 01:38:14)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Local Memory: 32 KB
 Device - Constant Memory: 1048576 KB
 Device - Global Memory: 5 GB
 Device - Max Buffer/Image Size: 1024 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [412]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [413]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [414]:
cl_program_files = [
    '../../Library/worker/sha256.cl',
    '../../Library/worker/msisdn.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: hash_main;get_single_hash;find_msisdn


## Single hash

In [415]:
# the text to hash
plaintext = '0722252318'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

# the hash output
hash_output = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

# print the results
print("Plaintext: %s" % plaintext)
print("CL result      : %s" % hexlify(hash_output))
print("Correct result : %s" % hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest()))
print("CL output      : %s" % hash_output)
print("CL output      : %s" % np.frombuffer(hash_output, dtype=np.uint8))
print("CL output      : %s" % hash_output.tobytes())

assert hexlify(hash_output) == b'2b59d1a6d1d1efaea83eea2b3b68c95770b4193a498bdfecedda80e936e9611c'

Plaintext: 0722252318
CL result      : b'2b59d1a6d1d1efaea83eea2b3b68c95770b4193a498bdfecedda80e936e9611c'
Correct result : b'2b59d1a6d1d1efaea83eea2b3b68c95770b4193a498bdfecedda80e936e9611c'
CL output      : [2798737707 2934952401  736771752 1472817211  974763120 3974073161
 3917535981  476178742]
CL output      : [ 43  89 209 166 209 209 239 174 168  62 234  43  59 104 201  87 112 180
  25  58  73 139 223 236 237 218 128 233  54 233  97  28]
CL output      : b'+Y\xd1\xa6\xd1\xd1\xef\xae\xa8>\xea+;h\xc9Wp\xb4\x19:I\x8b\xdf\xec\xed\xda\x80\xe96\xe9a\x1c'


In [416]:
hash_output

array([2798737707, 2934952401,  736771752, 1472817211,  974763120,
       3974073161, 3917535981,  476178742], dtype=uint32)

In [417]:
hash_string = '2b59d1a6d1d1efaea83eea2b3b68c95770b4193a498bdfecedda80e936e9611c'
np.frombuffer(unhexlify(hash_string), dtype=np.uint32)

array([2798737707, 2934952401,  736771752, 1472817211,  974763120,
       3974073161, 3917535981,  476178742], dtype=uint32)

## Find MSISDN

In [418]:
# get the target hash
hash_string = '2b59d1a6d1d1efaea83eea2b3b68c95770b4193a498bdfecedda80e936e9611c'
hash_input = np.frombuffer(unhexlify(hash_string), dtype=np.uint32)
hash_output = np.zeros(8, dtype=np.uint32)

# candidates
output_candidates_len = np.zeros(1, dtype=np.uint8)
output_candidates = np.zeros(255, dtype=np.uint32)

# allocate the memory for the variables on the device
# cl_hash_input = cl.Buffer(
#     cl_ctx,
#     cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
#     hostbuf=hash_input)

cl_hash_input = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

cl_output_candidates_len = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_WRITE | cl.mem_flags.USE_HOST_PTR,
    hostbuf=output_candidates_len)

cl_output_candidates = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    output_candidates.nbytes)

# execute the program
cl_program.find_msisdn(
    cl_queue, (3,), None,
    cl_hash_input,
    cl_output_candidates_len,
    cl_output_candidates)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_input)
cl.enqueue_copy(cl_queue, output_candidates, cl_output_candidates)

print(f'hash_output : {hash_output}')
print(f'output_candidates_len : {output_candidates_len}')
print(f'output_candidates     : {output_candidates}')

hash_output : [2798737707 2934952401  736771752 1472817211  974763120 3974073161
 3917535981  476178742]
output_candidates_len : [0]
output_candidates     : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
